# Create YOLO Dataset

In [ ]:
import os
import zipfile
from pylabel import importer
import shutil

In [ ]:
import os
from pylabel import importer

def import_dataset(path_to_annotations, path_to_images, dataset_name):
    dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name=dataset_name)
    return dataset

def split_and_export(dataset, output_path):
    # dataset.splitter.GroupShuffleSplit(train_pct=.6, val_pct=.2, test_pct=.2)
    dataset.export.ExportToYoloV5(output_path=output_path, yaml_file='dataset.yaml', copy_images=True)

# For train dataset
train_annotations = "COCOAnnotations/trainnew.json"
train_images = os.getcwd() + "/MergeDataset/"
train_dataset = import_dataset(train_annotations, train_images, "BCCD_coco")
split_and_export(train_dataset, 'YOLO_dataset_train/labels')

# For test dataset
test_annotations = "COCOAnnotations/testnew.json"
test_images = os.getcwd() + "/MergeDataset/"  # Assuming test images are in the same folder as train images
test_dataset = import_dataset(test_annotations, test_images, "BCCD_coco")
split_and_export(test_dataset, 'YOLO_dataset_test/labels')


In [ ]:
import os
import shutil
from pathlib import Path

# Define source and destination paths
test_source = 'YOLO_dataset_test/labels'
train_source = 'YOLO_dataset_train/labels'
test_dest = 'YOLO_dataset/labels/test'
train_dest = 'YOLO_dataset/labels/train'
source_file = "COCOAnnotations/dataset.yaml"
destination_dir = "YOLO_dataset"

# Create main directories
os.makedirs('YOLO_dataset/images/train', exist_ok=True)
os.makedirs('YOLO_dataset/images/test', exist_ok=True)

# Copy images from test and train directories
shutil.copytree('YOLO_dataset_test/images', 'YOLO_dataset/images/test', dirs_exist_ok=True)
shutil.copytree('YOLO_dataset_train/images', 'YOLO_dataset/images/train', dirs_exist_ok=True)

# Copy label directories
try:
    shutil.copytree(test_source, test_dest)
    shutil.copytree(train_source, train_dest)
    print("Directories copied successfully!")
except FileExistsError:
    print("Destination directories already exist. Remove or rename them and try again.")
except Exception as e:
    print(f"An error occurred: {e}")

# Copy dataset.yaml file
shutil.copy(source_file, destination_dir)

shutil.rmtree('YOLO_dataset_test')
shutil.rmtree('YOLO_dataset_train')

In [ ]:
# for k, v in model.named_parameters():
    # print(k)

# Train Using Ultralytics Library

In [ ]:
import os
from ultralytics import YOLO

# ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset'
# ROOT_DIR = os.getcwd() + '\YOLO_dataset'
ROOT_DIR = os.getcwd() + '/yolo-data'


# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('yolov5nu.pt')
# model = YOLO('yolov5s6u.pt')
# model = YOLO('yolo5s.pt')
# model = YOLO('yolov8n.pt')
# model = YOLO('yolov8s.pt')
# model = YOLO('yolov8m.pt')
# model = YOLO('yolov8l.pt')
# model = YOLO('yolov8x.pt')


# Freeze
# freeze = [f'model.{x}.' for x in range(20)]  # layers to freeze
# for k, v in model.named_parameters():
#     v.requires_grad = True  # train all layers
#     if any(x in k for x in freeze):
#         # print(f'freezing {k}')
#         v.requires_grad = False


# Use the model
# results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=2,project='saved_models/YOLO/',device='cuda',batch=4,workers=10)  # train the model
results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=2,project='saved_models/YOLO/',device='mps',batch=32,workers=10)  # train the model

# Train Using Ultralytics repo

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# !python3 yolov5/train.py --data /Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset/dataset.yaml --weights yolov5s.pt --epochs 100 --batch 4 --freeze 10


!python train.py --batch 128 --epochs 50 \
                 --data /Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset/dataset.yaml \
                 --hyp /Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset/dataset.yaml \
                 --weights yolov5l.pt \
                 --project  saved_models/YOLO/ \
                 --name YOLOv5_Final_Validation

# Training from list of models

In [ ]:
import os
from ultralytics import YOLO

# ROOT_DIR = '/Users/rishabhshah/Desktop/AIPI590/Object-Detection/YOLO_dataset'
ROOT_DIR = os.getcwd() + '\YOLO_dataset'



model_list = [
    # 'yolov5nu.pt',
    # 'yolov5s6u.pt',
    # 'yolov6-n.pt',
    # 'yolov6-n.pt',
    # 'yolov8n.pt',
    # 'yolov8s.pt',
    # 'yolov8m.pt',
    'yolov9c.pt'
]

for model_name in model_list:

    if os.path.exists(model_name.split(".")[0]):
        print("Folder exists")
        continue

    model = YOLO(model_name) 
    # Freeze
    try:
        freeze = [f'model.{x}.' for x in range(15)]  # layers to freeze
        for k, v in model.named_parameters():
            v.requires_grad = True  # train all layers
            if any(x in k for x in freeze):
                # print(f'freezing {k}')
                v.requires_grad = False
    except:
        model = YOLO(model_name) 


    # Use the model
    results = model.train(data=os.path.join(ROOT_DIR, "dataset.yaml"), epochs=500,project='saved_models/YOLO/',device='cuda',batch=8,workers=10,name=model_name.split('.')[0])  # train the model

In [ ]:
import os
from ultralytics import YOLO


model_list = [
    'yolov5nu.pt',
    'yolov5s6u.pt',
    # 'yolov6n.pt',
    # 'yolov6s.pt',
    'yolov8n.pt',
    'yolov8s.pt',
    'yolov8m.pt',
    'yolov9c.pt'
]


# Perform validation
for model_name in model_list:
    model = YOLO(f'saved_models/YOLO/{model_name.split(".")[0]}/weights/best.pt')
    results = model.val()

In [ ]:
# model.predict()